<a id="readme-top"></a>

# LP Presolve Testground

## Table of Contents
- [Table of Contents](#table-of-contents)
- [Introduction](#introduction)
- [Packages](#packages)
- [Modified LPProblem Structs](#modified-lpproblem-structs)
- [Test Problems](#test-problems)
- [Presolve Features](#presolve-features)
    - [Removing Empty Rows](#removing-empty-rows)
    - [Removing Empty Columns](#removing-empty-columns)
    - [Remove Linear Dependent Rows](#remove-linear-dependent-rows)
- [General Presolve Routine](#general-presolve-routine)
- [References](#references)

<p align="right">(<a href="#readme-top">back to top</a>)</p>

## Introduction

Presolving is a vital task of high performance LP solvers as it minimizes the size of the working problem.

**Procedure Presolve**
- Remove all fixed variables.
- **repeat**
    - **Check rows:**
        - Remove all row singletons.
        - Remove all forcing constraints.
        - Remove all dominated constraints.
    - **Check columns:**
        - Remove all free, implied free column singletons and all column singletons in combination with a doubleton equation.
        - Remove all dominated columns.
    - **Check for duplicates:**
        - Remove duplicate rows.
        - Remove duplicate columns.
- **until** no reductions in last pass.
- Remove all empty rows and columns.

**end Presolve.**
<sup id="cite1">[Anderson and Anderson, 1995](#ref1)</sup>



<p align="right">(<a href="#readme-top">back to top</a>)</p>

## Packages

```julia
import Pkg; Pkg.add("SuiteSparseQR")

In [ ]:
using LinearAlgebra
using SparseArrays
using SuiteSparse
using Random
using ArgParse

# local modules
push!(LOAD_PATH, realpath("../code"))
using lp_constants
using lp_utils
using lp_problem
using lp_read_mps

using BenchmarkTools
using Test

<p align="right">(<a href="#readme-top">back to top</a>)</p>

## Modified LPProblem Structs
Below I am experimenting with different formulations of the LPProblem to capture sufficient information for post-processing a reduced LP problem.

In [ ]:
# struct LPProblem
#     is_minimize::Bool  # True if the objective is to minimize
#     c::Vector{Float64}  # Objective function coefficients
#     A::SparseMatrixCSC{Float64, Int64}  # Constraint matrix
#     b::Vector{Float64}  # Right-hand side of constraints
#     l::Vector{Float64}  # Variable lower bounds
#     u::Vector{Float64}  # Variable upper bounds
#     vars::Vector{String}  # Variable names
#     constraint_types::Vector{Char}  # Constraint types
# end

In [ ]:
# # Define a struct to represent a Linear Programming problem
# struct LPProblemAdvannced
#     is_minimize::Bool  # True if the objective is to minimize
#     c::Vector{Float64}  # Objective function coefficients
#     A::SparseMatrixCSC{Float64, Int64}  # Constraint matrix
#     b::Vector{Float64}  # Right-hand side of constraints
#     l::Vector{Float64}  # Variable lower bounds
#     u::Vector{Float64}  # Variable upper bounds
#     vars::Vector{String}  # Variable names
#     constraint_types::Vector{Char}  # Constraint types
#     removed_rows::Union{Nothing, Vector{Int}}  # Optional field to store indices of removed rows
#     removed_cols::Union{Nothing, Vector{Int}}  # Optional field to store indices of removed columns (if needed)
# end


In [ ]:
struct PreprocessedLPProblem
    original_problem::LPProblem  # The original LP problem before preprocessing
    reduced_problem::LPProblem   # The reduced LP problem after preprocessing
    removed_rows::Vector{Int}    # Indices of removed rows
    removed_cols::Vector{Int}    # Indices of removed columns (if applicable)
    row_ratios::Dict{Int, Tuple{Int, Float64}}  # Mapping of removed rows to their corresponding row and ratio
end


<p align="right">(<a href="#readme-top">back to top</a>)</p>

## Test Problems

Here are some test problems that are used to check various aspects of the presolver.

In [ ]:
mps_filename = "../../benchmarks/mps_files/test.mps"
lp = read_mps_from_file(mps_filename)
print(lp)

In [ ]:
# This function custom entries data into the LPProblem struct.
function create_test_problem()
    is_minimize = true
    c = [1.0, 2.0, 0.0, 0.0, 3.0]
    A = sparse([1.0 2.0 0.0 0.0 3.0; 4.0 5.0 0.0 0.0 6.0])
    b = [7.0, 8.0]
    l = [0.0, 0.0, 0.0, 0.0, 0.0]
    u = [10.0, 10.0, 10.0, 10.0, 10.0]
    vars = ["x1", "x2", "x3", "x4", "x5"]
    constraint_types = ['=', '=']

    return LPProblem(is_minimize, c, A, b, l, u, vars, constraint_types)
end

In [ ]:
function create_test_problem_with_empty_rows()
    is_minimize = true
    c = [1.0, 2.0, 0.0, NaN, 3.0]
    A = sparse([0.0 0.0 0.0 0.0 0.0; 4.0 5.0 0.0 NaN NaN; 0.0 0.0 0.0 0.0 0.0])  # First and third rows are zero
    b = [0.0, 7.0, 0.0]  # Simulate an empty entry in the right-hand side
    l = [0.0, 0.0, 0.0, NaN, 0.0]
    u = [10.0, 10.0, NaN, 10.0, 10.0]
    vars = ["x1", "x2", "x3", "x4", "x5"]
    constraint_types = ['=', '=', '=']

    return LPProblem(is_minimize, c, A, b, l, u, vars, constraint_types)
end


In [ ]:
function create_test_problem_linear_dependence_rows()
    # Define the LP problem
    c = [1.0, 1.0]
    
    # Define the constraint matrix A with multiple sets of linear dependencies
    A = sparse([
        1.0 2.0;   # Row 1
        1.0 2.0;   # Row 2 (1 * Row 1)
        1.0 1.0;   # Row 3
        3.0 6.0;   # Row 4 (3 * Row 1)
        0.5 1.0;   # Row 5 (0.5 * Row 1)
        2.0 2.0;   # Row 6
        4.0 4.0;   # Row 7 (2 * Row 6)
        0.5 0.5;   # Row 8 (0.25 * Row 6)
        1.5 3.0;   # Row 9 (1.5 * Row 1)
        5.0 10.0   # Row 10 (5 * Row 1)
    ])
    
    # Define the corresponding right-hand side vector b
    b = [5.0, 5.0, 3.0, 15.0, 2.5, 4.0, 8.0, 2.0, 7.5, 25.0]
    
    l = [0.0, 0.0]
    u = [10.0, 10.0]
    vars = ["x1", "x2"]
    constraint_types = ['=', '=', '=', '=', '=', '=', '=', '=', '=', '=']
    
    return LPProblem(true, c, A, b, l, u, vars, constraint_types)
end


<p align="right">(<a href="#readme-top">back to top</a>)</p>

## Presolve Features


### Removing Empty Rows

This is the first and simplest presolving feature.


In [ ]:
function lp_remove_zero_rows(lp::LPProblem; ε::Float64=1e-8)
    # Find non-zero rows
    non_zero_rows = [i for i in 1:size(lp.A, 1) if any(abs.(lp.A[i, :]) .> ε)]
    removed_rows = setdiff(1:size(lp.A, 1), non_zero_rows)
    
    # Initialize the dictionary to store the ratios of removed rows
    row_ratios = Dict{Int, Tuple{Int, Float64}}()

    # Assign a ratio of 0.0 for all removed rows to indicate no values
    for removed_row in removed_rows
        row_ratios[removed_row] = (removed_row, 0.0)
    end

    # Create new LPProblem with non-zero rows
    new_A = lp.A[non_zero_rows, :]
    new_b = lp.b[non_zero_rows]
    new_constraint_types = lp.constraint_types[non_zero_rows]

    # Construct the reduced LPProblem
    reduced_lp = LPProblem(lp.is_minimize, lp.c, new_A, new_b, lp.l, lp.u, lp.vars, new_constraint_types)

    # Return the PreprocessedLPProblem struct containing the original and reduced LPProblem, removed rows, and ratios
    return PreprocessedLPProblem(lp, reduced_lp, removed_rows, Int[], row_ratios)
end

In [ ]:
# Create the test problem
lp = create_test_problem_with_empty_rows()

# Run the function to remove zero rows and preprocess the LP problem
preprocessed_lp = lp_remove_zero_rows(lp)

# Display the reduced problem and the removed rows with their ratios
println("Reduced A matrix:")
display(preprocessed_lp.reduced_problem.A)

println("Reduced b vector:")
display(preprocessed_lp.reduced_problem.b)

println("Removed rows and their ratios:")
for (removed_row, (base_row, ratio)) in preprocessed_lp.row_ratios
    println("Row $removed_row was removed. It is assigned a ratio of $ratio.")
end



### Removing Empty Columns

In [ ]:
# Function to remove empty columns from the LPProblem
function lp_remove_zero_columns(lp::LPProblem; ε::Float64=1e-8)
    # Find non-zero columns
    non_zero_columns = [j for j in 1:size(lp.A, 2) if any(abs.(lp.A[:, j]) .> ε)]
    removed_columns = setdiff(1:size(lp.A, 2), non_zero_columns)
    #removed_columns = Int32.(setdiff(1:size(lp.A, 2), non_zero_columns))

    # Create new LPProblem with without non-zero columns
    new_c = lp.c[non_zero_columns]
    new_A = lp.A[:, non_zero_columns]
    new_l = lp.l[non_zero_columns]
    new_u = lp.u[non_zero_columns]
    new_vars = lp.vars[non_zero_columns]

    # Construct the reduced LPProblem
    reduced_lp = LPProblem(lp.is_minimize, new_c, new_A, lp.b, new_l, new_u, new_vars, lp.constraint_types)

    # Return the PreprocessedLPProblem struct containing the original and reduced LPProblem and the removed rows
    return PreprocessedLPProblem(lp, reduced_lp, Int[], removed_columns)  # Empty vector for removed columns as we haven't modified columns

    # Return the reduced LPProblem and the list of removed column indices
    #return LPProblem(lp.is_minimize, new_c, new_A, lp.b, new_l, new_u, new_vars, lp.constraint_types), removed_column
end


In [ ]:
function post_process_remove_zero_columns(lp::LPProblem, reduced_solution::Vector{Float64}, removed_columns::Vector{Int})
    # Create a full solution vector initialized with zeros
    full_solution = zeros(Float64, length(lp.c))
    
    # Fill in the solution values for the non-zero columns
    non_zero_columns = setdiff(1:length(lp.c), removed_columns)
    full_solution[non_zero_columns] = reduced_solution
    
    return full_solution
end

In [ ]:
# function test_post_process()
#     # Create the original test problem
#     lp = create_test_problem()

#     # Apply presolve to remove zero columns
#     reduced_lp, removed_columns = lp_remove_zero_columns(lp)
    
#     # Mock solution of the reduced problem (for demonstration)
#     reduced_solution = [1.0, 2.0, 3.0]  # Example solution after solving the reduced problem
    
#     # Post-process to restore the full solution
#     full_solution = post_process_remove_zero_columns(lp, reduced_lp, removed_columns)
    
#     println("Full solution (after post-processing): ", full_solution)
# end

# # Run the test
# test_post_process()

### Remove Linear Dependent Rows


In [ ]:
function lp_remove_linearly_dependent_rows(lp::LPProblem; ε::Float64=1e-8)
        # Create the augmented matrix [A b]
        augmented_matrix = hcat(lp.A, lp.b)
    
        rows_to_check = collect(1:size(augmented_matrix, 1))  # Start with all rows to check
        removed_rows = Vector{Int}()  # List of removed rows
        row_ratios = Dict{Int, Tuple{Int, Float64}}()  # Store ratios of removed rows
        
        while length(rows_to_check) > 1
            current_row_index = rows_to_check[1]
            current_row = augmented_matrix[current_row_index, :]
            
            for i in rows_to_check[2:end]
                compare_row = augmented_matrix[i, :]
                
                # Find the first non-zero index in the current_row
                non_zero_idx = findfirst(abs.(current_row) .> ε)
                
                if non_zero_idx !== nothing && compare_row[non_zero_idx] != 0
                    ratio = current_row[non_zero_idx] / compare_row[non_zero_idx]
                    
                    # Check if multiplying the compare_row by ratio gives the current_row
                    if all(abs.(current_row .- ratio .* compare_row) .< ε)
                        push!(removed_rows, i)  # Mark the row for removal
                        row_ratios[i] = (current_row_index, ratio)  # Store the row index and ratio
                    end
                end
            end
            
            rows_to_check = setdiff(rows_to_check, [current_row_index; removed_rows])  # Remove current and dependent rows
        end
        
        # Create the reduced matrix and vectors by excluding the removed rows
        non_removed_rows = setdiff(1:size(lp.A, 1), removed_rows)
        reduced_A = lp.A[non_removed_rows, :]
        reduced_b = lp.b[non_removed_rows]
        reduced_constraint_types = lp.constraint_types[non_removed_rows]
        
        # Construct the reduced LPProblem
        reduced_lp = LPProblem(
            lp.is_minimize,
            lp.c,
            reduced_A,
            reduced_b,
            lp.l,
            lp.u,
            lp.vars,
            reduced_constraint_types
        )
        
        # Return the PreprocessedLPProblem struct
        return PreprocessedLPProblem(lp, reduced_lp, removed_rows, Int[], row_ratios)
    end

In [ ]:
# Create the test problem
lp_test = create_test_problem_linear_dependence_rows()

# Run the function to preprocess the LP problem
preprocessed_lp = lp_remove_linearly_dependent_rows(lp_test)

# Display the reduced LP problem's A matrix and b vector
println("Reduced A matrix:")
display(preprocessed_lp.reduced_problem.A)

println("Reduced b vector:")
display(preprocessed_lp.reduced_problem.b)

println("Removed rows and their ratios:")
for (removed_row, (base_row, ratio)) in preprocessed_lp.row_ratios
    println("Row $removed_row was removed. It is a multiple of Row $base_row with a ratio of $ratio.")
end



### Tightening Constraints 

In [ ]:
function remove_forcing_constraints(lp::LPProblem; ε::Float64=1e-8)
    A, b, l, u = lp.A, lp.b, lp.l, lp.u
    removed_rows = Int[]
    row_ratios = Dict{Int, Tuple{Int, Float64}}()

    # Iterate over each constraint
    for i in 1:size(A, 1)
        row = A[i, :]
        lb, ub = 0.0, 0.0  # Bounds based on the type of constraint

        if lp.constraint_types[i] == '<' || lp.constraint_types[i] == '≤'
            ub = b[i]
        elseif lp.constraint_types[i] == '>' || lp.constraint_types[i] == '≥'
            lb = b[i]
        elseif lp.constraint_types[i] == '='
            lb = b[i]
            ub = b[i]
        end

        # Calculate the minimum and maximum possible values of the row
        min_row_value = sum(row[j] * (row[j] > 0 ? l[j] : u[j]) for j in 1:size(A, 2))
        max_row_value = sum(row[j] * (row[j] > 0 ? u[j] : l[j]) for j in 1:size(A, 2))

        # Check if the constraint is forcing
        if (ub != 0 && max_row_value <= ub + ε) || (lb != 0 && min_row_value >= lb - ε)
            push!(removed_rows, i)
            row_ratios[i] = (i, 0.0)
        end
    end

    # Create the reduced A matrix and b vector by removing forcing constraints
    non_removed_rows = setdiff(1:size(A, 1), removed_rows)
    reduced_A = A[non_removed_rows, :]
    reduced_b = b[non_removed_rows]
    reduced_constraint_types = lp.constraint_types[non_removed_rows]

    # Construct the reduced LPProblem
    reduced_lp = LPProblem(lp.is_minimize, lp.c, reduced_A, reduced_b, lp.l, lp.u, lp.vars, reduced_constraint_types)

    return PreprocessedLPProblem(lp, reduced_lp, removed_rows, Int[], row_ratios)
end


In [ ]:
# Example usage with dummy data
lp_test = LPProblem(true,
                    [1.0, 2.0],
                    sparse([1.0 0.0; 0.0 1.0; 1.0 1.0]),
                    [4.0, 2.0, 5.0],
                    [0.0, 0.0],
                    [10.0, 10.0],
                    ["x1", "x2"],
                    ['≤', '≥', '='])

preprocessed_lp = remove_forcing_constraints(lp_test)

# Display the reduced problem
println("Reduced A matrix:")
display(preprocessed_lp.reduced_problem.A)

println("Reduced b vector:")
display(preprocessed_lp.reduced_problem.b)

println("Removed rows and their ratios:")
for (removed_row, (base_row, ratio)) in preprocessed_lp.row_ratios
    println("Row $removed_row was removed. It has a ratio of $ratio indicating it was a forcing constraint.")
end

<p align="right">(<a href="#readme-top">back to top</a>)</p>

## General Presolve Routine

Currently this is just copy and pasted from Harleys lp_llama_ipm.ipynb notebook.

In [ ]:
function presolve(lp::LPProblem; eps::Float64=1e-8)
    is_minimize, c, A, b = lp.is_minimize, lp.c, lp.A, lp.b
    l, u = copy(lp.l), copy(lp.u)  # Create copies to avoid modifying the original bounds
    vars, constraint_types = lp.vars, lp.constraint_types
    m, n = size(A)

    # Initialize masks for rows and columns to keep
    keep_rows = trues(m)
    keep_cols = trues(n)

    # Step 1: Remove zero columns
    for j in 1:n
        if nnz(A[:, j]) == 0 && abs(c[j]) < eps
            keep_cols[j] = false
        end
    end

    # Step 2: Remove zero rows
    for i in 1:m
        if nnz(A[i, :]) == 0
            if abs(b[i]) < eps
                keep_rows[i] = false
            else
                error("Infeasible problem: zero row with non-zero RHS")
            end
        end
    end

    # Step 3: Remove duplicate rows
    for i in 1:m
        if !keep_rows[i]
            continue
        end
        for j in (i+1):m
            if !keep_rows[j]
                continue
            end
            if A[i, :] == A[j, :] && abs(b[i] - b[j]) < eps && constraint_types[i] == constraint_types[j]
                keep_rows[j] = false
            elseif A[i, :] == -A[j, :] && abs(b[i] + b[j]) < eps &&
                    ((constraint_types[i] == 'L' && constraint_types[j] == 'G') ||
                     (constraint_types[i] == 'G' && constraint_types[j] == 'L'))
                keep_rows[j] = false
            end
        end
    end

    # Step 4 - Conservative bound tightening with feasibility checks - k is the selected column
    bound_updates = true
    iteration = 1
    while bound_updates
        bound_updates = false
        println("\nBound tightening loop: iteration $iteration")

        lower_bounds = [Float64[] for _ in 1:n]
        upper_bounds = [Float64[] for _ in 1:n]  

        for i in 1:m  # Loop over constraints
            if !keep_rows[i]
                continue
            end

            println("\nAnalyzing bounds for constraint $i:")
            row = A[i, :]  # Get the i-th row            

            for k in 1:n  # select variable column
                if (A[i,k] == 0) || !keep_cols[k]
                    continue
                end

                lower_sum = sum((l[j] * aij for (j, aij) in zip(row.nzind, row.nzval) if j != k && aij > 0), init=0.0)
                upper_sum = sum((u[j] * aij for (j, aij) in zip(row.nzind, row.nzval) if j != k && aij < 0), init=0.0)
                new_bound = (b[i] - lower_sum - upper_sum)/ A[i,k]
                println("  analyzing bounds for variable $(vars[k]): k = $(k): A[i,k] = $(A[i,k]), b[i] = $(b[i]), lower_sum = $lower_sum, upper_sum = $upper_sum, new_bound = $new_bound")
                
                if constraint_types[i] == 'L'
                    if A[i,k] > 0
                        push!(upper_bounds[k], new_bound)
                    else
                        push!(lower_bounds[k], new_bound)
                    end
                elseif constraint_types[i] == 'G'
                    if A[i,k] > 0
                        push!(lower_bounds[k], new_bound)
                    else
                        push!(upper_bounds[k], new_bound)
                    end
                elseif constraint_types[i] == 'E'
                    push!(lower_bounds[k], new_bound)
                    push!(upper_bounds[k], new_bound)
                end

                if !isempty(lower_bounds[k])
                    if length(lower_bounds[k]) > 0
                        new_lower = maximum(lower_bounds[k])
                        if new_lower > l[k] + eps
                            println("variable $(vars[k]) tightening lower bound: $new_lower (prev: $(l[k]))")
                            l[k] = new_lower
                            bound_updates = true                    
                        end
                    end
                end
                
                if !isempty(upper_bounds)
                    if length(upper_bounds[k]) > 0
                        new_upper = minimum(upper_bounds[k])
                        if new_upper < u[k] - eps
                            println("variable $(vars[k]) tightening upper bound: $new_upper (prev: $(u[k]))")
                            u[k] = new_upper
                            bound_updates = true
                        end
                    end
                end
                
                # Ensure lower bound doesn't exceed upper bound
                if l[k] > u[k] + eps
                    println("WARNING: variable $(vars[k]) lower bound ($(l[k])) exceeds upper bound ($(u[k])) for $(vars[k]) - adjusting bounds to maintain feasibility")
                    avg_bound = (l[k] + u[k]) / 2
                    l[k] = avg_bound - eps
                    u[k] = avg_bound + eps
                end
                println("calculated bounds for $(vars[k]): [$(l[k]), $(u[k])]")                
            end
        end
           
        if bound_updates
            iteration += 1
        else
            println("No further bound updates possible")
        end
    end

    # Step 5: Fix variables and tighten bounds
    fixed_vars = Dict{String, Float64}()
    for j in 1:n
        col = A[:, j]
        if nnz(col) == 1
            i = findfirst(!iszero, col)
            if abs(col[i]) == 1 && constraint_types[i] == 'E'
                val = b[i] / col[i]
                if l[j] <= val && val <= u[j]
                    fixed_vars[vars[j]] = val
                    keep_cols[j] = false
                    b .-= val * col
                end
            end
        end
    end

    # Apply the reductions
    A_new = A[keep_rows, keep_cols]
    b_new = b[keep_rows]
    c_new = c[keep_cols]
    l_new = l[keep_cols]
    u_new = u[keep_cols]
    vars_new = vars[keep_cols]
    constraint_types_new = constraint_types[keep_rows]

    # Adjust the objective for fixed variables
    obj_adjust = 0.0
    for (j, var) in enumerate(vars)
        if haskey(fixed_vars, var)
            obj_adjust += c[j] * fixed_vars[var]
        end
    end

    if options[OPTION_VERBOSE]
        println("\nPresolve summary:")
        println("  Original problem size: $(m) x $(n)")
        println("  Reduced problem size: $(sum(keep_rows)) x $(sum(keep_cols))")
        println("  Number of fixed variables: $(length(fixed_vars))")
        println("  Objective adjustment: $obj_adjust")
        println("  Updated bounds:")
        for (var, lb, ub) in zip(vars_new, l_new, u_new)
            println("    $var: [$lb, $ub]")
        end
    end

    return LPProblem(is_minimize, c_new, Matrix(A_new), b_new, l_new, u_new, vars_new, constraint_types_new), fixed_vars, obj_adjust
end

<p align="right">(<a href="#readme-top">back to top</a>)</p>

## Junk to be cleaned latter.

In [ ]:
# function postsolve(lp::LPProblem, fixed_vars::Dict{String, Float64}, obj_adjust::Float64, original_shape::Tuple{Int, Int}, reduced_solution::Vector{Float64})
#     m, n = original_shape
#     solution = zeros(Float64, n)

#     # Place the reduced solution into the correct positions
#     j_reduced = 1
#     for j in 1:n
#         if haskey(fixed_vars, lp.vars[j])
#             solution[j] = fixed_vars[lp.vars[j]]
#         else
#             solution[j] = reduced_solution[j_reduced]
#             j_reduced += 1
#         end
#     end

#     # Adjust the objective value with the stored adjustment
#     adjusted_objective_value = sum(lp.c .* solution) + obj_adjust

#     return solution, adjusted_objective_value
# end


In [ ]:
# function simple_presolve(lp::LPProblem, ε::Float64=1e-8)
#     is_minimize, c, A, b = lp.is_minimize, lp.c, lp.A, lp.b
#     l, u = copy(lp.l), copy(lp.u)  # Create copies to avoid modifying the original bounds
#     vars, constraint_types = lp.vars, lp.constraint_types
#     m, n = size(A)

#     # Store the original shape
#     original_shape = (m, n)

#     # Initialize a mask for rows to keep
#     keep_rows = trues(m)

#     # Step 1: Remove empty rows
#     for i in 1:m
#         if nnz(A[i, :]) == 0
#             if abs(b[i]) < ε
#                 keep_rows[i] = false
#             else
#                 error("Infeasible problem: empty row with non-zero RHS")
#             end
#         end
#     end

#     # Apply the reductions
#     A_new = A[keep_rows, :]
#     b_new = b[keep_rows]
#     constraint_types_new = constraint_types[keep_rows]

#     # Output the number of removed rows
#     # println("\nSimple Presolve summary:")
#     # println("  Original number of rows: $m")
#     # println("  Number of rows after removal: $(sum(keep_rows))")
#     # println("  Removed rows: $(m - sum(keep_rows))")

#     return LPProblem(is_minimize, c, Matrix(A_new), b_new, l, u, vars, constraint_types_new), original_shape
# end


In [ ]:
# mps_filename::String = "../benchmarks/mps_files/nug04.mps"

# # read the mps file
# lp = read_mps_from_file(mps_filename)

In [ ]:
# # Assuming you have an LPProblem instance `lp` and you applied the simple presolve:
# lp_reduced, original_shape = presolve(lp)

# # Solve the reduced LP problem (using any LP solver) to get `reduced_solution`

# # Then use simple_postsolve to obtain the solution and objective value in the original problem's space
# #solution, adjusted_objective_value = simple_postsolve(lp_reduced, original_shape, lp_reduced)
# println(lp_reduced)



In [ ]:
# # Assuming you have an LPProblem instance `lp`:
# lp_reduced, fixed_vars, obj_adjust, original_shape = presolve_with_shape(lp)

# # Solve the reduced LP problem (using any LP solver) to get `reduced_solution`

# # Then use postsolve to map the reduced solution back to the original problem's space
# solution, adjusted_objective_value = postsolve(lp, fixed_vars, obj_adjust, original_shape, reduced_solution)

In [ ]:
# lp_reduced = presolve_with_shape(lp)
# lp_post = postsolve(lp_reduced)

<p align="right">(<a href="#readme-top">back to top</a>)</p>

## References

1. <a id="ref1"></a> Andersen, E.D., Andersen, K.D. Presolving in linear programming. *Mathematical Programming* 71, 221–245 (1995). [https://doi.org/10.1007/BF01586000](https://doi.org/10.1007/BF01586000)